# 1D diffusion simulation

**See [https://github.com/boutproject/boutcore-examples](https://github.com/boutproject/boutcore-examples) for links to the interactive version.**

## Import boutcore

In [ ]:
import boutcore as bc
# This might fail if boutcore is not installed, or not in 
# your PYTHONPATH. This can happen if the mpi module it has
# been installed for is not loaded.
from numpy import sqrt, sin
from boutcore import DDX, D2DX2, PhysicsModel

## Define a simple model

In [ ]:
class Diffusion(PhysicsModel):
    def init(self, restart):
        self.mesh = bc.Mesh.getGlobal()
        self.n = bc.Field3D.fromMesh(self.mesh)

        self.solve_for(n=self.n)
        self.source = bc.create3D("exp(-(x-0.5)^2/4)")
        self.D = 0.01

    def rhs(self, time):
        # Run communications
        self.mesh.communicate(self.n)
        
        # Set the time derivative by adding/... to it
        # make sure to never overwrite it
        # ddt_n = bla does NOT set the time derivative
        # Waiting for https://github.com/dschwoerer/BOUT-dev/issues/40
        ddt_n = self.n.ddt()
        ddt_n.set(0)
        ddt_n += self.D * D2DX2(self.n)
        # Add some perturbation
        ddt_n += self.source * sin(time)


## Run the simulation

In [ ]:
# NB: We need to restart the kernel to restart BOUT++ / switch folder
bc.init("-d diffusion")

In [ ]:
diffusion = Diffusion()
print("Starting the simulation")
# Sorry, for now no output
# https://github.com/boutproject/BOUT-dev/issues/2354
diffusion.solve()
print("The simulation is finished")

## Do the post processing

In [ ]:
import xbout
data = xbout.open_boutdataset(
    datapath="diffusion/BOUT.dmp.*.nc",
    inputfilepath="diffusion/BOUT.inp",
    info=False,
).squeeze(drop=True)

In [ ]:
data

In [ ]:
data.n.plot()